This notebook outlines the figure production for everything current except the data contained in the 2020 PL Decennial Census (population, high level race and ethnicity, and housing occupancy), and the historical and population projections.

###### Imports, Settings, Functions

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import cycler
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
from functools import reduce
import matplotlib as mpl
from matplotlib.axis import Axis
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
#functions
def prettypercentchange(x, y):
    return round((x - y)*100/y, 1).astype(str)+'%'
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def percent(x, y):
    return (x/y)*100

In [42]:
percentchange(1, 10)

-90.0

###### Read-In's

In [43]:
#2000
Tiger2000 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/TIGERS2000.csv').set_index(['NAME', 'GEO_ID'])
SufTiger2000 = Tiger2000.add_suffix(' 2000')
SF12000 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/2000SF1.csv').set_index(['NAME', 'GEO_ID'])
SufSF12000 = SF12000.add_suffix(' 2000')
SF32000 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/2000SF3.csv').set_index(['NAME', 'GEO_ID'])
SufSF32000 = SF32000.add_suffix(' 2000')
dfs2000 = [Tiger2000, SF12000, SF32000]
twothousand= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2000)
twothousand.columns = pd.MultiIndex.from_product([twothousand.columns, ['2000'], ['None']])
#2010
SF12010 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/2010SF1.csv').set_index(['NAME', 'GEO_ID'])
SufSF12010 = SF12010.add_suffix(' 2010')
Tiger2010 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/TIGERS2010.csv').set_index(['NAME', 'GEO_ID'])
SufTiger2010 = Tiger2010.add_suffix(' 2010')
Subject2010 = pd.read_feather('../../../../GNRC-Data-Integration/Outputs/2010Subject').set_index(['NAME', 'GEO_ID'])
SufSubject2010 = Subject2010.add_suffix(' 2010')
ACS52010 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/2010ACS5YR.csv').set_index(['NAME', 'GEO_ID'])
SufACS52010 = ACS52010.add_suffix(' 2010')
dfs2010 = [Tiger2010, SF12010, Subject2010, ACS52010]
twothousandten = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2010)
twothousandten.columns = pd.MultiIndex.from_product([twothousandten.columns, ['2010'], ['None']])
#2020/2021
#the PL is a stand-in so that the change metrics will run
PL2021 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/2020PL.csv').set_index(['NAME', 'GEO_ID'])
SufPL2021 = PL2021.add_suffix(' 2021')
ACS52021 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/2021ACS5YR.csv').set_index(['NAME', 'GEO_ID'])
SufACS52021 = ACS52021.add_suffix(' 2021')
Tiger2021 = pd.read_csv('../../../../GNRC-Data-Integration/Outputs/TIGERS2020.csv').set_index(['NAME', 'GEO_ID'])
SufTiger2021 = Tiger2021.add_suffix(' 2021')
Subject2021 = pd.read_feather('../../../../GNRC-Data-Integration/Outputs/2021Subject').set_index(['NAME', 'GEO_ID'])
SufSubject2021 = Subject2021.add_suffix(' 2021')
dfs2021 = [PL2021, Subject2021, ACS52021, Tiger2021]
twothousandtwentyone = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)
twothousandtwentyone.columns = pd.MultiIndex.from_product([twothousandtwentyone.columns, ['2021'], ['None']])
#all years tidy
dfs = [twothousand, twothousandten, twothousandtwentyone]
dftidy = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs)
#population densities tidy
dftidy[('Population Per Square Mile', '2000', 'None')] = dftidy[('Population', '2000', 'None')]/dftidy[('Square Miles', '2000', 'None')]
dftidy[('Population Per Square Mile', '2010', 'None')] = dftidy[('Population', '2010', 'None')]/dftidy[('Square Miles', '2010', 'None')]
dftidy[('Population Per Square Mile', '2021', 'None')] = dftidy[('Population', '2021', 'None')]/dftidy[('Square Miles', '2021', 'None')]
dftidy[('Population Per Square Acre', '2000', 'None')] = dftidy[('Population', '2000', 'None')]/dftidy[('Square Acres', '2000', 'None')]
dftidy[('Population Per Square Acre', '2010', 'None')] = dftidy[('Population', '2010', 'None')]/dftidy[('Square Acres', '2010', 'None')]
dftidy[('Population Per Square Acre', '2021', 'None')] = dftidy[('Population', '2021', 'None')]/dftidy[('Square Acres', '2021', 'None')]
dftidy[('Housing Units Per Square Mile', '2000', 'None')] = dftidy[('Occupancy:Total Households', '2000', 'None')]/dftidy[('Square Miles', '2000', 'None')]
dftidy[('Housing Units Per Square Mile', '2010', 'None')] = dftidy[('Occupancy:Total Households', '2010', 'None')]/dftidy[('Square Miles', '2010', 'None')]
dftidy[('Housing Units Per Square Mile', '2021', 'None')] = dftidy[('Occupancy:Total Households', '2021', 'None')]/dftidy[('Square Miles', '2021', 'None')]
dftidy[('Housing Units Per Square Acre', '2000', 'None')] = dftidy[('Occupancy:Total Households', '2000', 'None')]/dftidy[('Square Acres', '2000', 'None')]
dftidy[('Housing Units Per Square Acre', '2010', 'None')] = dftidy[('Occupancy:Total Households', '2010', 'None')]/dftidy[('Square Acres', '2010', 'None')]
dftidy[('Housing Units Per Square Acre', '2021', 'None')] = dftidy[('Occupancy:Total Households', '2021', 'None')]/dftidy[('Square Acres', '2021', 'None')]
#all years suffix
dfs = [SufTiger2000, SufSF12000, SufSF32000, SufSF12010, SufTiger2010, SufSubject2010,
       SufACS52010, SufPL2021, SufACS52021, SufTiger2021, SufSubject2021]
dfsuff = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'GEO_ID'], how='outer'), dfs)
#population densities suffix
dfsuff['Population per Square Mile 2000'] = dfsuff['Population 2000']/dfsuff['Square Miles 2000']
dfsuff['Population per Square Acre 2000'] = dfsuff['Population 2000']/dfsuff['Square Acres 2000']
dfsuff['Population per Square Mile 2010'] = dfsuff['Population 2010']/dfsuff['Square Miles 2010']
dfsuff['Population per Square Acre 2010'] = dfsuff['Population 2010']/dfsuff['Square Acres 2010']
dfsuff['Population per Square Mile 2021'] = dfsuff['Population 2021']/dfsuff['Square Miles 2021']
dfsuff['Population per Square Acre 2021'] = dfsuff['Population 2021']/dfsuff['Square Acres 2021']
#placeholder
dfsuffmerged = dfsuff
dftidymerged = dftidy

In [44]:
geos = ['Sumner County', 'Gallatin', 'Millersville', 'Goodlettsville', 'Hendersonville', 'Portland', 
        'Westmoreland', 'Mitchellville', 'White House']

In [45]:
#suff
dfsuff = dfsuffmerged.reset_index()
dfsuff = dfsuff.set_index(['NAME']).transpose()

In [46]:
dfsuff = dfsuff[geos]

In [47]:
dfsuff = dfsuff.transpose()
cols = list(dfsuff.columns)
cols.remove("GEO_ID")
dfsuff[cols] = dfsuff[cols].astype(float)

In [48]:
dfsuff = dfsuff.reset_index()

In [49]:
#tidy
dftidy = dftidymerged

In [50]:
#Make a list of all columns with a percent sign in them so that we can loop through all of the OTHER columns
allcols = list(dftidy.columns)
lilguy = '%'
nopercent = list(filter(lambda x: lilguy not in x, (i[0] for i in allcols)))

In [51]:
def delete__by_values(lst, values):
    values_as_set = set(values)
    return [ x for x in lst if x not in values_as_set ]

In [52]:
removing = ['Group Quarters:Population','Group Quarters:Institutionalized','Group Quarters:Institutionalized, Adults in Correctional Facility',
            'Group Quarters:Institutionalized, Juvenile Facility','Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility',
            'Group Quarters:Institutionalized, Other Facility','Group Quarters:Noninstitutionalized',
            'Group Quarters:Noninstitutionalized, College or University Student Housing','Group Quarters:Noninstitutionalized, Military Quarters',
            'Group Quarters:Noninstitutionalized, Other','HealthCoverage:Total Series','HealthCoverage:None','HealthCoverage: With Healthcare Coverage',
            'HealthCoverage: With Public Healthcare Coverage', 'HealthCoverage: With Private Healthcare Coverage',
            'StructureAge:Total Structures', 'StructureAge:Built 2014 or Later', 'StructureAge:Built 2010 to 2013', 'StructureAge:Built 2000 to 2009',
            'StructureAge:Built 1990 to 1999', 'StructureAge:Built 1980 to 1989', 'StructureAge:Built 1970 to 1979', 'StructureAge:Built 1960 to 1969',
            'StructureAge:Built 1950 to 1959', 'StructureAge:Built 1940 to 1949','StructureAge:Built 1939 or Earlier', 'StructureAge:Median Year Structure Built',
            'StructureAge:Median Age of Structure','Commute:Walked:Less than 10 Minutes','Commute:Walked:10 to 14 Minutes', 'Commute:Walked:15 to 19 Minutes',
            'Commute:Walked:20 to 24 Minutes','Commute:Walked:25 to 29 Minutes','Commute:Walked:30 to 34 Minutes', 'Commute:Walked:35 to 44 Minutes',
            'Commute:Walked:45 to 59 Minutes','Commute:Walked:60 or More Minutes', 'Commute:Taxi, Motorcycle, Bicycle, Other:10 to 14 Minutes',
            'Commute:Taxi, Motorcycle, Bicycle, Other:15 to 19 Minutes', 'Commute:Taxi, Motorcycle, Bicycle, Other:20 to 24 Minutes',
            'Commute:Taxi, Motorcycle, Bicycle, Other:25 to 29 Minutes', 'Commute:Taxi, Motorcycle, Bicycle, Other:30 to 34 Minutes',
            'Commute:Taxi, Motorcycle, Bicycle, Other:35 to 44 Minutes', 'Commute:Taxi, Motorcycle, Bicycle, Other:45 to 59 Minutes',
            'Commute:Taxi, Motorcycle, Bicycle, Other:60 or More Minutes', 'Disability:Total Population', 'Disability:With Disability', 
            'Disability:Hearing Difficulty', 'Disability:Vision Difficulty', 'Disability:Cognitive Difficulty', 'Disability:Ambulatory Difficulty',
            'Disability:Self-Care Difficulty', 'Disability:Independent Living Difficulty', 'Disability:Total Disabilities', 
            'Disability:Population with One Disability', 'Disability:Population with Two or More Disabilities', 'Foreign Born:Entered 2010 or Later',
            'Foreign Born: Entered 2000 to 2009', 'Foreign Born: Entered 1990 to 1999', 'Foreign Born: Entered Before 1990', 'Disability Tally']

In [53]:
colsnopercentages = delete__by_values(nopercent, removing)

In [54]:
for i in colsnopercentages:
    dftidy[('{} % Change'.format(i), 'None','2000-2021')] = percentchange(dftidy[('{}'.format(i),'2021','None')], dftidy[('{}'.format(i),'2000','None')])
    dftidy[('{} % Change'.format(i), 'None','2000-2010')] = percentchange(dftidy[('{}'.format(i),'2010','None')], dftidy[('{}'.format(i),'2000','None')])
    dftidy[('{} % Change'.format(i), 'None','2010-2021')] = percentchange(dftidy[('{}'.format(i),'2021','None')], dftidy[('{}'.format(i),'2010','None')])
    dftidy[('{} Change'.format(i), 'None','2000-2021')] = (dftidy[('{}'.format(i),'2021','None')] - dftidy[('{}'.format(i),'2000','None')])
    dftidy[('{} Change'.format(i), 'None','2000-2010')] = (dftidy[('{}'.format(i),'2010','None')] - dftidy[('{}'.format(i),'2000','None')])
    dftidy[('{} Change'.format(i), 'None','2010-2021')] = (dftidy[('{}'.format(i),'2021','None')] - dftidy[('{}'.format(i),'2010','None')])

KeyError: 24537

In [55]:
#stack and rename to get tidy data for both year and time period - half of dataframe will be null depending on if it's a change or point in time metric
dftidy = dftidy.stack([1, 1])
dftidy = dftidy.reset_index(drop = False)
dftidy = dftidy.rename(columns = {'level_1':'GEO_ID', 'level_2':'Year', 'level_3':'Time Frame'})

In [56]:
#filter
dftidy = dftidy.set_index('NAME')
dftidy = dftidy.transpose()

In [57]:
dftidy = dftidy[geos]

In [58]:
dftidy = dftidy.transpose()

In [59]:
dftidy = dftidy.reset_index(drop = False).rename(columns = {'index':'NAME'})

In [60]:
dftidy['Year'] = dftidy['Year'].replace({'None':0})

In [61]:
cols = list(dftidy.columns)
cols.remove("GEO_ID")
cols.remove('Time Frame')
cols.remove('NAME')
dftidy[cols] = dftidy[cols].astype(float)

###### Set Style Parameters

In [62]:
#HARDCODE COLORS AND FONTS TO BE PASSED IN OTHER PARAMETERS
GNRCDARKBLUE = '#00263E'
GNRCLIGHTBLUE = '#6DB0C7'
GNRCORANGE = '#EB7525'
GNRCCOOLGRAY = '#7B7E85'
PINK = '#DBA1A2'
BURNTORANGE = '#C7846D'
TEAL = '#6DC7B1'
GREEN = '#B0C76D'
PURPLE = '#C76DB0'
TABLECOL1 = '#E1EFF2'
TABLECOL2 = '#F4F9F9'
GNRCFONTONE = 'Calibri'
GNRCFONTTWO = 'DIN Condensed'
GNRCFONTTHREE = 'Palatino Linotype'
#FONT PARAMETERS
SMALLER_SIZE = 10
SMALL_SIZE = 11
MEDIUM_SIZE = 13
BIGGER_SIZE = 18
BIGGEST_SIZE = 20
rcParams['font.size'] = SMALL_SIZE           #controls default text sizes
rcParams['font.weight'] = 'normal'           #controls default text weight
rcParams['axes.titlesize'] = BIGGER_SIZE      #fontsize of the axes title
rcParams['axes.labelsize'] = MEDIUM_SIZE     #fontsize of the x and y labels
rcParams['xtick.labelsize'] = SMALL_SIZE     #fontsize of the xtick labels
rcParams['ytick.labelsize'] = SMALL_SIZE     #fontsize of the ytick labels
rcParams['legend.fontsize'] = SMALL_SIZE   #legend fontsize
rcParams['font.family'] = GNRCFONTONE        #although calibri is not a font "family" this defaults to the font itself
rcParams['axes.facecolor'] = 'None'   
#FIGURE PARAMETERS
rcParams['figure.figsize'] = 10, 5           #figure size in inches
rcParams['figure.facecolor'] = 'None'        #color of figure, none makes it transparent
rcParams['figure.edgecolor'] = 'None'        #color of figure edges
rcParams['figure.autolayout'] = True         #tells matplotlib to optimize figure layout within parameters
rcParams['figure.dpi'] = 500                 #even though we save as a vector file this renders the image nicely here and for github
#AXES PARAMETERS
#Axes contain most of the figure elements
rcParams['axes.grid'] = False                 #display grid
rcParams['axes.grid.which'] = 'major'        #only display major grid
rcParams['axes.axisbelow'] = True            #draw axis gridlines and ticks below patches
rcParams['text.color'] = GNRCDARKBLUE        #axes text color
rcParams['axes.labelcolor'] = GNRCDARKBLUE   #axes label color
rcParams['xtick.color'] = GNRCCOOLGRAY       #axes x tick label color
rcParams['ytick.color'] = GNRCCOOLGRAY       #axes y tick label color
rcParams['axes.labelpad'] = 15               #the padding, or distance, between the label and figure
rcParams['axes.titlepad'] = 20               #the padding, or distance, between the title and figure
#AXIS LABEL FORMATTERS
yearformat = '%.0f'                          #takes float datatype to look like a year
numbernodecimalformat = '{x:,.0f}'           #takes float datatype to a number with no decimal with commas at the thousandth
numberonedecimalformat = '{x:,.1f}'          #takes float datatype to a number with one decimal with commas at the thousandth
numbertwodecimalformat = '{x:,.2f}'          #takes float datatype to a number with two decimal with commas at the thousandth
percentformat = '{x:,.1f}%'                  #takes float datatype to a number with one decimal with commas at the thousandth and an appended %
percentnodecimalformat = '{x:,.0f}%'         #takes float datatype to a number with one decimal with commas at the thousandth and an appended %
moneyformat = '${x:,.0f}'                    #takes float datatype to a number with no decimal with commas at the thousandth and a MONEY sign
#GRID PARAMETERS
rcParams['grid.color'] = GNRCCOOLGRAY        #grid color
rcParams['grid.linestyle'] = '-'             #solid linestyle
rcParams['grid.alpha'] = '0.7'               #transparency
#LINE PLOT PARAMETERS
rcParams['lines.linewidth'] = 1.5            #line width
rcParams['lines.linestyle'] = '-'            #solid linestyle
rcParams['lines.marker'] = 'o'               #circular marker
linealpha = 0.8                              #no rcparameter for line transparency, so set here for line and called in plotting function
#ALL FIGURE COLOR CYCLER
#set colors for default where it's max 2 lines in a chart for this example
#color dictionary mapping specific geos to colors will have to be executed in the cell
rcParams['axes.prop_cycle'] = cycler(color = [GNRCORANGE, GNRCCOOLGRAY, GNRCLIGHTBLUE, GNRCDARKBLUE, PINK, BURNTORANGE, TEAL, GREEN, PURPLE])
#TAKE AWAY BOX FOR ALL FIGURES
rcParams.update({'axes.spines.top': False, 'axes.spines.right': False, 'axes.spines.bottom': False, 'axes.spines.left': False})
#TICK MARKS - you can choose to show major versus minor or no ticks in the code
rcParams['xtick.major.size'] = 8
rcParams['xtick.minor.size'] = 4
rcParams['ytick.major.size'] = 6
rcParams['ytick.minor.size'] = 3
rcParams['ytick.minor.size'] = 3
#TICKS
tickaxisselection = 'both'                  #both axes have ticks
majororminortick = 'major'                  #only major ticks selected for now, although we just established a default size for the minor if we want them
#LEGEND
rcParams['legend.framealpha'] = 0.8         #transparency of legend
rcParams['legend.facecolor'] = 'inherit'    #legend inherits the facecolor of the figure
rcParams['legend.frameon'] = False          #no frame for legend

In [63]:
#table color functions
def howwide(width):
    ['None' for x in range(width)]
def howlong(indexlength):
    ["None" for x in range(indexlength)]
def tablecolors(indexlength, width):
    if indexlength == 2:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)]]
    if indexlength == 3:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)]]
    if indexlength == 4:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)]]
    if indexlength == 5:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)]]
    if indexlength == 6:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 7:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)]]
    if indexlength == 8:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 9:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)]]
    if indexlength == 10:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 11:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)]]
    if indexlength == 12:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 13:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)]]
    if indexlength == 14:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 15:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)]]
    if indexlength == 16:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 17:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)]]
    if indexlength == 18:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 19:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)]]
    if indexlength == 20:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 21:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)]]
    if indexlength == 22:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 23:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)]]
    if indexlength == 24:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)]]
    if indexlength == 25:
        return [[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)],[TABLECOL1 for x in range(width)],[TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)],
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], [TABLECOL1 for x in range(width)], [TABLECOL2 for x in range(width)], 
               [TABLECOL1 for x in range(width)]]

In [64]:
#table formatting for non-merged table function
col1 = '#00263E'
col2 = '#E1EFF2'
edge1 = 'vertical'
edge2 = 'horizontal'
def allheaders(col1, col2, edge1, edge2):
    for cell in table.get_children():
        cell_text = cell.get_text().get_text()
        if cell_text not in row_headers\
        and cell_text not in column_headers:
            cell.set_edgecolor(col1)
        else:
            cell.set(edgecolor=col2)
    for cell in table.get_children():
        cell_text = cell.get_text().get_text()
        if cell_text in column_headers:
            cell.visible_edges = edge1
        elif cell_text in row_headers:
            cell.visible_edges = edge2
    for cell in table.get_children():
        cell_text = cell.get_text().get_text()
        if cell_text in row_headers\
        or cell_text in column_headers:
            cell.set_text_props(fontproperties=FontProperties(weight='bold'))
#table formatting for merged table function
def allheadersmerged(col1, col2, edge1, edge2):
    for cell in table.get_children():
        cell_text = cell.get_text().get_text()
        if cell_text not in row_headers\
        and cell_text not in doublecols:
            cell.set_edgecolor(col1)
        elif cell_text in column_headers:
            cell.set_edgecolor(None)
        else:
            cell.set(edgecolor=col2)    
        if cell_text in column_headers:
            cell.visible_edges = 'B'
        elif cell_text in row_headers:
            cell.visible_edges = edge2
        elif cell_text in col_names:
            cell.visible_edges = 'LR'
        if cell_text in row_headers\
        or cell_text in doublecols:
            cell.set_text_props(fontproperties=FontProperties(weight='bold'))
#function to make figure invisible for tables
guy1 = False
guy2 = 'off'
guy3 = 'tight'
def makeinvisible(guy1, guy2, guy3):            
    return fig.patch.set_visible(guy1), ax.axis(guy2), ax.axis(guy3)
#data labels function
def singledatalabels(xoffset, yoffset, labelformat):
    for j, k in zip(x, y):
        label = labelformat.format(k)
        plt.annotate(label, (j, k), textcoords = 'offset points', xytext = (xoffset, yoffset), ha = 'center')
#double data labels function       
def doubledatalabels(xoffset1, yoffset1, xoffset2, yoffset2, labelformat, size):
    for j, k in zip(xpos, y1):
        label = labelformat.format(k)
        ax.annotate(label, (j, k), textcoords = 'offset points', xytext = (xoffset1, yoffset1), fontsize = size, ha = 'center')
    for j, k in zip(xpos, y2):
        label = labelformat.format(k)
        ax.annotate(label, (j, k), textcoords = 'offset points', xytext = (xoffset2, yoffset2), fontsize = size, ha = 'center')
#merge two cells function        
def mergecells(table, ix0, ix1):
    ix0,ix1 = np.asarray(ix0), np.asarray(ix1)
    d = ix1 - ix0
    if not (0 in d and 1 in np.abs(d)):
        raise ValueError("ix0 and ix1 should be the indices of adjacent cells. ix0: %s, ix1: %s" % (ix0, ix1))
    if d[0]==-1:
        edges = ('BRL', 'TRL')
    elif d[0]==1:
        edges = ('TRL', 'BRL')
    elif d[1]==-1:
        edges = ('BTR', 'BTL')
    else:
        edges = ('BTL', 'BTR')
    # hide the merged edges
    for ix,e in zip((ix0, ix1), edges):
        table[ix[0], ix[1]].visible_edges = e
    txts = [table[ix[0], ix[1]].get_text() for ix in (ix0, ix1)]
    tpos = [np.array(t.get_position()) for t in txts]
    # center the text of the 0th cell between the two merged cells
    trans = (tpos[1] - tpos[0])/2
    if trans[0] > 0 and txts[0].get_ha() == 'right':
        # reduce the transform distance in order to center the text
        trans[0] /= 2
    elif trans[0] < 0 and txts[0].get_ha() == 'right':
        # increase the transform distance...
        trans[0] *= 2
    txts[0].set_transform(mpl.transforms.Affine2D().translate(*trans))
    # hide the text in the 1st cell
    txts[1].set_visible(False)
#merge multiple cells function
def mergemultcells(table, cells):
    cells_array = [np.asarray(c) for c in cells]
    h = np.array([cells_array[i+1][0] - cells_array[i][0] for i in range(len(cells_array) - 1)])
    v = np.array([cells_array[i+1][1] - cells_array[i][1] for i in range(len(cells_array) - 1)])
    # if it's a horizontal merge, all values for `h` are 0
    if not np.any(h):
        # sort by horizontal coord
        cells = np.array(sorted(list(cells), key=lambda v: v[1]))
        edges = ['BTL'] + ['BT' for i in range(len(cells) - 2)] + ['BTR']
    elif not np.any(v):
        cells = np.array(sorted(list(cells), key=lambda h: h[0]))
        edges = ['TRL'] + ['RL' for i in range(len(cells) - 2)] + ['BRL']
    else:
        raise ValueError("Only horizontal and vertical merges allowed")
    for cell, e in zip(cells, edges):
        table[cell[0], cell[1]]   
#stacked bar labels function
belowstackedbarlabels = -5
abovestackedbarlabels = 2
def stackedbarlabels(locationoffset):
    for bar in ax.patches:
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + bar.get_y() + locationoffset, str(float(round(bar.get_height(), 1))) + '%', ha='center')
#rounded boxes for stacked bars
boxstyleinput = "round,pad=-0.060,rounding_size=0.15"
def roundedboxes(boxstyle):
    new_patches = []
    for patch in reversed(ax.patches):
        bb = patch.get_bbox()
        color=patch.get_facecolor()
        p_bbox = FancyBboxPatch((bb.xmin, bb.ymin),abs(bb.width), abs(bb.height),boxstyle=boxstyleinput, ec="none", fc=color, mutation_aspect=1)
        patch.remove()
        new_patches.append(p_bbox)
    for patch in new_patches:
        ax.add_patch(patch) 

In [88]:
#set primary geo for colors
primarygeo = 'Millersville'
#quick geo filters
reggeos = dftidy.loc[(dftidy['NAME'] == 'Sumner County')|
                     (dftidy['NAME'] == 'Gallatin')|(dftidy['NAME'] == 'Goodlettsville')|(dftidy['NAME'] == 'Hendersonville')|
                     (dftidy['NAME'] == 'Millersville')|(dftidy['NAME'] == 'Mitchellville')|(dftidy['NAME'] == 'Portland')|(dftidy['NAME'] == 'Westmoreland')|
                     (dftidy['NAME'] == 'White House')].reset_index(drop = True)
reggeost = dfsuff.loc[(dfsuff['NAME'] == 'Sumner County')|
                      (dfsuff['NAME'] == 'Gallatin')|(dfsuff['NAME'] == 'Goodlettsville')|(dfsuff['NAME'] == 'Hendersonville')|(dfsuff['NAME'] == 'Millersville')|
                      (dfsuff['NAME'] == 'Mitchellville')|(dfsuff['NAME'] == 'Portland')|(dfsuff['NAME'] == 'Westmoreland')|
                      (dfsuff['NAME'] == 'White House')].reset_index(drop = True)
reggeosnocounty = dftidy.loc[(dftidy['NAME'] == 'Gallatin')|(dftidy['NAME'] == 'Goodlettsville')|(dftidy['NAME'] == 'Hendersonville')|
                             (dftidy['NAME'] == 'Millersville')|(dftidy['NAME'] == 'Mitchellville')|(dftidy['NAME'] == 'Portland')|(dftidy['NAME'] == 'Westmoreland')|
                             (dftidy['NAME'] == 'White House')].reset_index(drop = True)
reggeosnocountyt = dfsuff.loc[(dfsuff['NAME'] == 'Gallatin')|(dfsuff['NAME'] == 'Goodlettsville')|(dfsuff['NAME'] == 'Hendersonville')|
                              (dfsuff['NAME'] == 'Millersville')|(dfsuff['NAME'] == 'Mitchellville')|(dfsuff['NAME'] == 'Portland')|
                              (dfsuff['NAME'] == 'Westmoreland')|(dfsuff['NAME'] == 'White House')].reset_index(drop = True)
Primary = dftidy.loc[dftidy['NAME'] == 'Millersville']
Primaryt = dfsuff.loc[dfsuff['NAME'] == 'Millersville']
County = dftidy.loc[dftidy['NAME'] == 'Sumner County']
Countyt = dfsuff.loc[dfsuff['NAME'] == 'Sumner County']
Gallatin = dftidy.loc[dftidy['NAME'] == 'Gallatin']
Gallatint = dfsuff.loc[dfsuff['NAME'] == 'Gallatin']
Goodlettsville = dftidy.loc[dftidy['NAME'] == 'Goodlettsville']
Goodlettsvillet = dfsuff.loc[dfsuff['NAME'] == 'Goodlettsville']
Hendersonville = dftidy.loc[dftidy['NAME'] == 'Hendersonville']
Hendersonvillet = dfsuff.loc[dfsuff['NAME'] == 'Hendersonville']
Millersville = dftidy.loc[dftidy['NAME'] == 'Millersville']
Millersvillet = dfsuff.loc[dfsuff['NAME'] == 'Millersville']
Mitchellville = dftidy.loc[dftidy['NAME'] == 'Mitchellville']
Mitchellvillet = dfsuff.loc[dfsuff['NAME'] == 'Mitchellville']
Portland = dftidy.loc[dftidy['NAME'] == 'Portland']
Portlandt = dfsuff.loc[dfsuff['NAME'] == 'Portland']
Westmoreland = dftidy.loc[dftidy['NAME'] == 'Westmoreland']
Westmorelandt = dfsuff.loc[dfsuff['NAME'] == 'Westmoreland']
WhiteHouse = dftidy.loc[dftidy['NAME'] == 'White House']
WhiteHouset = dfsuff.loc[dfsuff['NAME'] == 'White House']

# Check Stats

City Trends 2010-19  
47% increase in median household income  
76% reduction in unemployed civilians  
80% reduction in unemployment rate  
% increase in high income households ($200k)  
54% decrease in population in poverty  

In [97]:
data = Primary
cols = ['NAME', 'Year', 'Time Frame', 'Median Household Income', 'Median Household Income % Change', 'Median Household Income Change', 'Employment:Unemployed Civilian', 
        'Employment:Unemployed Civilian % Change', 'Employment:Unemployed Civilian Change', 'Employment%:Population 16 Over that are Unemployed Civilians', 
        'HHIncome:200K or More', 'HHIncome:200K or More % Change', 'HHIncome:200K or More Change', 'Poverty: Population Below Poverty Level', 
       'Poverty: Population Below Poverty Level % Change', 'Poverty: Population Below Poverty Level Change', 'Poverty%: Below Poverty Level', 'HHIncome:150 to 199,999']

In [98]:
data = data[cols]
data = data.loc[(data['Time Frame'] != '2000-2021')& (data['Time Frame'] != '2000-2010')]

In [99]:
data.head(6)

,NAME,Year,Time Frame,Median Household Income,Median Household Income % Change,Median Household Income Change,Employment:Unemployed Civilian,Employment:Unemployed Civilian % Change,Employment:Unemployed Civilian Change,Employment%:Population 16 Over that are Unemployed Civilians,HHIncome:200K or More,HHIncome:200K or More % Change,HHIncome:200K or More Change,Poverty: Population Below Poverty Level,Poverty: Population Below Poverty Level % Change,Poverty: Population Below Poverty Level Change,Poverty%: Below Poverty Level,"HHIncome:150 to 199,999"
12,Millersville,2000.0,None,41573.0,NaN,NaN,96.0,NaN,NaN,2.554550,0.0,NaN,NaN,586.0,NaN,NaN,11.816899,26.0
13,Millersville,2010.0,None,44477.0,NaN,NaN,450.0,NaN,NaN,9.176183,0.0,NaN,NaN,890.0,NaN,NaN,14.563901,12.0
14,Millersville,2021.0,None,61687.0,NaN,NaN,106.0,NaN,NaN,2.119152,19.0,NaN,NaN,505.0,NaN,NaN,8.280046,149.0
17,Millersville,0.0,2010-2021,NaN,38.694157,17210.0,NaN,-76.444444,-344.0,NaN,NaN,inf,19.0,NaN,-43.258427,-385.0,NaN,NaN
